W tym notatniku przejdę przez uczenie, walidację oraz predykcję modeli YOLOv8.

Duża część kodu jest zapisana w Markdown'ie, ponieważ output generowany przez takie komórki jest ogromny.

## Biblioteki

In [1]:
import os
from ultralytics import YOLO
from ultralytics.utils.benchmarks import benchmark
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import ast
from IPython.display import Video, display, HTML
from base64 import b64encode

In [2]:
import torch
try:
    torch.cuda.set_device(0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
except:
    device = "cpu"

## Tworzenie plików konfiguracyjnych yaml

Zacznijmy od utworzenia plików konfiguracyjnych (z rozszerzeniem .yaml).

Znajdują się w nich informację o lokalizacji datasetów oraz nazwy klas.

In [3]:
def create_config_yaml(file_name, path_dir, train_dir, val_dir, test_dir = None, classes_fn = None):
    if not os.path.isdir(path_dir):
        print(f"{path_dir} is not a directory")
        return None
    if not os.path.isdir(os.path.join(path_dir, train_dir)):
        print(f"{os.path.join(path_dir, train_dir)} is not a directory")
        return None
    if not os.path.isdir(os.path.join(path_dir, val_dir)):
        print(f"{os.path.join(path_dir, val_dir)} is not a directory")
        return None
    if test_dir is not None:
        if not os.path.isdir(os.path.join(path_dir, test_dir)):
            print(f"{os.path.join(path_dir, test_dir)} is not a directory")
            return None
    else:
        test_dir = ''
    if classes_fn is not None:
        if not os.path.isfile(classes_fn):
            print(f"Classes file {classes_fn} is not a file")
            return None
    else:
        print(f"Classes file can't be 'None'")
        return None
        

    with open(file_name, 'w') as f:
        f.write('\n')

        f.write('path: ')
        f.write(str(path_dir))
        f.write('\n')

        f.write('train: ')
        f.write(str(train_dir))
        f.write('\n')

        f.write('val: ')
        f.write(str(val_dir))
        f.write('\n')

        f.write('test: ')
        f.write(str(test_dir))
        f.write(' # Optional\n\n')

        f.write('# Classes\n')
        f.write('names: ' + '\n')
        with open(classes_fn, 'r') as f_classes:
            for i, line in enumerate(f_classes):
                f.write(f'  {i}: {line}')

#### Photos

In [ ]:
file_name = 'data/config_photos.yaml'
path_dir = 'C:/Users/Bartek/Documents/Studia/INZ/Program/yolov8_models/data'
train_dir = 'photos/train'
val_dir = 'photos/val'
test_dir = 'photos/test'
classes_fn = 'data/cards.names'

create_config_yaml(file_name, path_dir, train_dir, val_dir, test_dir, classes_fn)

#### Videos

In [ ]:
file_name = 'data/config_videos.yaml'
path_dir = 'C:/Users/Bartek/Documents/Studia/INZ/Program/yolov8_models/data'
train_dir = 'videos/train'
val_dir = 'videos/val'
test_dir = 'videos/test'
classes_fn = 'data/cards.names'

create_config_yaml(file_name, path_dir, train_dir, val_dir, test_dir, classes_fn)

## Uczenie modeli

Mając pliki konfiguracyjne oraz wcześniej wygenerowane datasety, możemy przejść do utworzenia modeli, omówienia parametrów oraz treningu.

### Utworzenie modeli

Utwórzmy nowe modele od zera (bazują one na architekturach już gotowych modeli, ale nie są pretrenowane). Modele te to *yolov8m*, czyli takie średniej wielkości.

```python
#####
model_photos = YOLO("yolov8m.yaml")

model_videos = YOLO("yolov8m.yaml")
#####
```

### Parametry i metryki trenowania

Zanim przejdziemy do trenowania, omówmy najważniejsze parametry występujące podczas treningu:

- **data** - Ścieżka do pliku konfiguracyjnego zestawu danych (np. coco8.yaml). Plik ten zawiera parametry specyficzne dla zbioru danych, w tym ścieżki do danych szkoleniowych i walidacyjnych, nazwy klas i liczbę klas.

- **epochs** - Całkowita liczba epok szkoleniowych. Każda epoka reprezentuje pełne przejście przez cały zestaw danych. Dostosowanie tej wartości może wpłynąć na czas trwania treningu i wydajność modelu.

- **project** - Nazwa katalogu projektu, w którym zapisywane są wyniki treningu. Umożliwia zorganizowane przechowywanie różnych eksperymentów.

- **name** - Nazwa treningu. Służy do tworzenia podkatalogu w folderze projektu, w którym przechowywane są dzienniki treningowe i wyniki.

- **seed** - Ustawia losowe ziarno dla treningu, zapewniając powtarzalność wyników między przebiegami z tymi samymi konfiguracjami.

- **batch** - Ilość obrazów przetwarzanych jednocześnie podczas treningu.

- **imgsz** - Docelowy rozmiar obrazu do treningu. Wszystkie obrazy są skalowane do tego wymiaru przed wprowadzeniem ich do modelu. Wpływa na dokładność modelu i złożoność obliczeniową.

- **workers** - Liczba wątków roboczych do ładowania danych. Wpływa na szybkość wstępnego przetwarzania danych i wprowadzania ich do modelu, co jest szczególnie przydatne w konfiguracjach z wieloma procesorami graficznymi.

- **lr0** - Początkowa szybkość uczenia się (dla optymizer: SGD=1E-2, Adam=1E-3). Dostosowanie tej wartości ma kluczowe znaczenie dla procesu optymalizacji, wpływając na szybkość aktualizacji wag modelu.

- **lrf** - Końcowy wskaźnik uczenia się jako ułamek wskaźnika początkowego = (lr0 * lrf), używany w połączeniu z harmonogramami w celu dostosowania wskaźnika uczenia się w czasie.

- **optimizer** - Wybór optymalizatora do szkolenia. Opcje obejmują SGD, Adam, AdamW, NAdam, RAdam, RMSProp itp. lub auto do automatycznego wyboru na podstawie konfiguracji modelu. Wpływa na szybkość i stabilność zbieżności.

- **momentum** - Współczynnik pędu dla SGD lub beta1 dla optymalizatorów Adam, wpływający na włączenie przeszłych gradientów do bieżącej aktualizacji.

- **amp** - Umożliwia automatyczne szkolenie z mieszaną precyzją (AMP), zmniejszając zużycie pamięci i prawdopodobnie przyspieszając szkolenie przy minimalnym wpływie na dokładność.

- **val** - Umożliwia walidację podczas treningu, pozwalając na okresową ocenę wydajności modelu na oddzielnym zbiorze danych.

- **patience** - Liczba epok, w których należy odczekać bez poprawy wskaźników walidacji przed wcześniejszym zatrzymaniem treningu. Pomaga zapobiegać nadmiernemu dopasowaniu, zatrzymując szkolenie, gdy wydajność osiąga plateau.

- **warmup_epochs** - Liczba epok dla rozgrzewki tempa uczenia, stopniowe zwiększanie tempa uczenia od niskiej wartości do początkowego tempa uczenia w celu wczesnego ustabilizowania treningu.

<br></br>

Dodatkowo przejdźmy przez metryki wykorzystane w uczeniu:
- **P (Precision)**: Dokładność wykrytych obiektów, wskazująca, ile wykryć było prawidłowych.

- **R (Recall)**: Zdolność modelu do identyfikacji wszystkich instancji obiektów na obrazach.

- **mAP50**: Średnia precyzja obliczona przy progu intersection over union (IoU) wynoszącym 0,50. Jest to miara dokładności modelu uwzględniająca tylko "łatwe" wykrycia.

- **mAP75**: Średnia precyzja obliczona przy progu intersection over union (IoU) wynoszącym 0,75.

- **mAP50-95**: Średnia średnia precyzja obliczona przy różnych progach IoU, w zakresie od 0,50 do 0,95. Daje to kompleksowy obraz wydajności modelu na różnych poziomach trudności wykrywania. Jest to "najważniejsza" metryka, ponieważ w oparciu o nią będziemy dokonywać oceny modelu oraz ona decyduje o zatrzymaniu uczenia modelu.

### Trenowanie

Zacznijmy trenowanie na dwóch wcześniej utworzonych datasetach (*photos* i *videos*). Trening odbędzie się w 2 sesjach z lekko zmenionymi parametrami.

Trening odbywa się na tzw. batch'ach, czyli połączonych obrazach (w ilości równej *batch_size*), na których zaznaczone są współrzędne informacji o karcie, wraz z indeksem klasy, z której jest dana karta. Poniżej przykład części batcha w dwóch obrazach:

<div style="display: flex;">
  <div style="text-align: center; width: 45%;">
    <img src='models/photos/train_/train_batch0.jpg' width='700' alt="Train Batch 1"/>
  </div>
  <div style="text-align: center; width: 45%;">
    <img src='models/photos/train_/train_batch1.jpg' width='700' alt="Train Batch 2"/>
  </div>
</div>

Natomiast po zakończeniu treningu przeprowadzamy walidację na zbiorze walidacyjnym. Obrazy z tamtąd też łączymy w batch'e i na nich określamy prawdopodobieństwo (tutaj zaokrąglane do jednego miejsca po przecinku), że to jest dana klasa:

<div style="display: flex;">
  <div style="text-align: center; width: 45%;">
    <img src='models/photos/train_/val_batch0_pred.jpg' width='700' alt="Val Batch 1"/>
  </div>
  <div style="text-align: center; width: 45%;">
    <img src='models/photos/train_/val_batch1_pred.jpg' width='700' alt="Val Batch 2"/>
  </div>
</div>

#### Photos

- Sesja 1

```python
model_photos.train(
    data = 'data/config_photos.yaml',
    epochs = 100,
    project = 'models/photos',
    name = 'train_',
    seed = 2024,
    batch = 0.8,    # 80% of GPU VRAM = 84
    imgsz = 640,
    save = True,
    workers = 8,
    plots = True,
    lr0 = 0.001,
    lrf = 0.01,
    momentum = 0.9,
    amp = True,
    val = True,
    optimizer = 'AdamW',
    patience = 10,
    warmup_epochs = 0,
    )
```


Zatrzymano po 69 epokach z powodu braku poprawy. Jako że **patience = 10**, to najlepsze wagi zaobserwowano w 59 epoce i to właśnie od niej rozpoczniemy sesję drugą uczenia.

- Sesja 2

Podczas sesji drugiej bierzemy niższy **lr0 = 0.0001** oraz **imgsz = 1024**, czyli uczymy wolniej na obrazach wyższej rozdzielczości.

```python
model_photos.train(
    data = 'data/config_photos.yaml',
    epochs = 100,
    project = 'models/photos',
    name = 'train_2',
    seed = 2024,
    batch = 0.8,    # 80% of GPU VRAM = 33
    imgsz = 1024,
    save = True,
    workers = 8,
    plots = True,
    lr0 = 0.0001,
    lrf = 0.01,
    momentum = 0.9,
    amp = True,
    val = True,
    optimizer = 'AdamW',
    patience = 10,
    warmup_epochs = 0,
    )
```


Ta sesja została zatrzymana po 18 epokach, więc nasz finalny model pochodzi z 8 epoki tego treningu.

<br></br>

#### Videos

- Sesja 1

```python
model_videos.train(
    data = 'data/config_videos.yaml',
    epochs = 100,
    project = 'models/videos',
    name = 'train_',
    seed = 2024,
    batch = 0.8,    # 80% of GPU VRAM = 84
    imgsz = 640,
    save = True,
    workers = 8,
    plots = True,
    lr0 = 0.001,
    lrf = 0.01,
    momentum = 0.9,
    amp = True,
    val = True,
    optimizer = 'AdamW',
    patience = 10,
    warmup_epochs = 0,
    )
```


Zatrzymano po 68 epokach, co oznacza, że najlepsze wagi otrzymaliśmy po 58 epoce. Tak samo jak w przypadku *photos* od niej wznowimy trening z nowymi parametrami.

- Sesja 2

Podczas sesji drugiej bierzemy niższy **lr0 = 0.0001** oraz **imgsz = 1024**, czyli uczymy wolniej na obrazach wyższej rozdzielczości.

```python
model_videos.train(
    data = 'data/config_videos.yaml',
    epochs = 100,
    project = 'models/videos',
    name = 'train_2',
    seed = 2024,
    batch = 0.8,    # 80% of GPU VRAM = 33
    imgsz = 1024,
    save = True,
    workers = 8,
    plots = True,
    lr0 = 0.0001,
    lrf = 0.01,
    momentum = 0.9,
    amp = True,
    val = True,
    optimizer = 'AdamW',
    patience = 10,
    warmup_epochs = 0,
    )
```


Zatrzymanie drugiej sesji nastąpiło po 70 epokach, więc finalny model to ten po 60 epokach treningu.

### Podsumowanie treningu (wykresy i opis)

Przejdźmy teraz do omówienia wyników treningu i ich interpretacji.

Najpierw jednak omówmy co oznaczają składniki ogólnej wartości strat:
- box_loss - mierzy błąd w przewidywaniu współrzędnych obwiedni. Zachęca model do dostosowania przewidywanych obwiedni do obwiedni rzeczywistych.

- cls_loss - określa ilościowo błąd w przewidywaniu klasy obiektu dla każdego obwiedni. Zapewnia to, że model dokładnie identyfikuje kategorię obiektu.

- dfl_loss - to wyspecjalizowany składnik strat, który pomaga poprawić wykrywanie obiektów w scenariuszach z nieostrymi lub rozmytymi obrazami. Zachęca model do skupienia się na poprawie wykrywania w takich trudnych warunkach.


W uczeniu wykorzystujemy 2 zbiory:
- train - zbiór treningowy, na którym nasz zbiór się uczy

- val - zbiór walidacyjny, dzięki któremu możemy uniknąć zjawiska overfitting'u oraz pomaga on nam we wcześniejszym przerwaniu treningu.

#### Photos

Zobaczmy najpierw wykresy rezultatu treningów:

<img src='notebooks/plots/photos/all_plots.png' width='1200' alt="Train Results Photos"/>

Widzimy, że trening przebiega dość sprawnie. Po przejściu do sesji 2 znacznie poprawiają się wartości opierające się na zbiorze validacyjnym co jest bardzo pożądane.

Teraz jeszcze rzućmy okiem na wykres 'metrics_mAP50-95':

<img src='notebooks/plots/photos/separated/metrics_mAP50-95(B).png' width='1000' alt="Train Precision Photos"/>

Maksymalna wartość to ponad **97%** na zbiorze walidacyjnym, co jest świetnym wynikiem.

#### Videos

Znomu zerknijmy na wszystkie wykresy:

<img src='notebooks/plots/videos/all_plots.png' width='1200' alt="Train Results Videos"/>

Uczenie na zbiorze treningowym pzebiega bardzo sprawnie, jednak przy przejściu do sesji 2 możemy zaobserwować znaczny wzrost wartości strat na zbiorze walidacyjnym

Oraz na wykres 'metrics_mAP50-95':

<img src='notebooks/plots/videos/separated/metrics_mAP50-95(B).png' width='1000' alt="Train Precision Videos"/>

Tutaj maksimum wynosi niecałe **91%**, będące nadal dobrym wynikiem.

#### Omówienie

Widzimy, że **photos** mają delikatnie lepszy model niż **videos**. Przejawia się to mniejszymi wartościami straty oraz lepszą średnią precyzją.

Pomimo, że parametry modeli były bardzo zbliżone, tak rezultaty są dość różne. Jest to spowodowane oczywiście różnymi zbiorami treningowymi i walidacyjnymi. 

Możemy wysnuć wniosek, że *model_photos* miał dane "lepszej" jakości i model jest bardziej stabilny.

### Model 'photos_large'

Korzystając z tego, że **photos** mają "lepsze" dane, utwórzmy model *large* opierający się na danych z **photos**.

Modele typu *large* charakteryzują się lepszą dokładnością i stabilnością, kosztem delikatnego opóźnienia oraz dłuższego czesu uczenia.

```python
#####
model_photos_large = YOLO("yolov8l.yaml")
#####
```

- Sesja 1
```python
model_photos_large.train(
    data = 'data/config_photos.yaml',
    epochs = 100,
    project = 'models/photos_large',
    name = 'train_',
    seed = 2024,
    batch = 0.9,    # 90% of GPU VRAM = 60
    imgsz = 640,
    save = True,
    workers = 8,
    plots = True,
    lr0 = 0.001,
    lrf = 0.01,
    momentum = 0.9,
    amp = True,
    val = True,
    optimizer = 'AdamW',
    patience = 20,
    )
```

Zatrzymano po 55 epokach z powodu braku poprawy. **patience = 20**, więc najlepsze wagi zaobserwowano w 35 epoce i to właśnie od niej rozpoczniemy sesję drugą uczenia.

- Sesja 2

Podczas sesji drugiej bierzemy niższy **lr0 = 0.0001** oraz **imgsz = 1024**, czyli uczymy wolniej na obrazach wyższej rozdzielczości. Dodatkowo zmniejszamy **patience = 10** oraz **epochs = 30**.

```python
model_photos_large.train(
    data = 'data/config_photos.yaml',
    epochs = 30,
    project = 'models/photos_large',
    name = 'train_2',
    seed = 2024,
    batch = 0.9,    # 90% of GPU VRAM = 23
    imgsz = 1024,
    save = True,
    workers = 8,
    plots = True,
    lr0 = 0.0001,
    lrf = 0.01,
    momentum = 0.9,
    amp = True,
    val = True,
    optimizer = 'AdamW',
    patience = 10,
    )
```

Zatrzymanie drugiej sesji nastąpiło po 15 epokach, więc finalny model to ten po 5 epokach treningu.

Spójrzmy na wykresy rezultatu treningów:

<img src='notebooks/plots/photos_large/all_plots.png' width='1200' alt="Train Results Photos Large"/>

Trening przebiega bardzo dobrze. Po rozpoczęciu na sesję 2 widzimy wzrost wartości strat (co jest spowodowane domyślną wartością *warmup_epochs = 3*), które szybko zaczynają opadać, a precyzja rośnie.

Teraz jeszcze rzućmy okiem na wykres 'metrics_mAP50-95':

<img src='notebooks/plots/photos_large/separated/metrics_mAP50-95(B).png' width='1000' alt="Train Precision Photos Large"/>

Maksymalna wartość to prawie **96%** na zbiorze walidacyjnym, co stanowi bardzo dobry wynik.

## Walidacja oraz testy modeli

Gdy tworzyliśmy nasze datasety to stanowiły one zbiory: treningowe, walidacyjne i testowe. Jak dotąd wykorzystywaliśmy tylko zbiory treningowe i walidacyjne. Do finalnej walidacji modeli oraz określenia, który z nich jest najlepszy wykorzystamy zbiory testowe. 

W celu uczciwego porównania modeli utworzymy jeden zbiór testowy łączący zbiory testowe **photos** i **videos**, czyli 30000 obrazów.

```python
val_photos = model_photos.val(split='test', plots=True, project='models/photos', name='val', half=False)
val_videos = model_videos.val(split='test', plots=True, project='models/videos', name='val', half=False)
val_photos_large = model_photos_large.val(split='test', plots=True, project='models/photos_large', name='val', half=False)
````

Po wykonaniu walidacji modeli na połączonym zbiorze testowym zapiszemy ich wyniki za pomocą funkcji:

```python
def return_metrics(results, save=False, save_dir='runs'):
    metrics = {}
    metrics['map'] = results.box.map             # map50-95
    metrics['map50'] = results.box.map50         # map50
    metrics['map75'] = results.box.map75         # map75
    metrics['maps'] = list(results.box.maps)     # a list contains map50-95 of each category
    metrics['mp'] = results.box.mp               # P
    metrics['mr'] = results.box.mr               # R

     if save:
     with open(os.path.join(save_dir, 'results.txt'), 'w') as file:
        file.write(str(metrics))

    return metrics


metrics_photos = return_metrics(val_photos, True, 'models/photos/val')
metrics_videos = return_metrics(val_videos, True, 'models/videos/val')
metrics_photos_large = return_metrics(val_photos_large, True, 'models/photos_large/val')
```

Po wykonaniu powyższego kodu możemy zobaczyć finalne wyniki walidacji.

In [6]:
def print_val_results(val_txt):
    with open(val_txt, 'r') as file:
        results = ast.literal_eval(file.read())

    print('mAP 50-95:', round(results['map'], 6))
    print('mAP 50:', round(results['map50'], 6))
    print('mAP 75:', round(results['map75'], 6))
    print('mP:', round(results['mp'], 6))
    print('mR:', round(results['mr'], 6))

    with open('data/cards.names', 'r') as f:
        cards = f.read().split("\n")
    cards = [c for c in cards if c!='']

    df_map = pd.DataFrame({'Karta': cards, 'mAP 50-95': results['maps']})

    return df_map

Dzięki tej funkcji zobaczymy średnie wartości metryk oraz najmniejsze i największe precyzje.

#### Photos

In [ ]:
df_map_photos = print_val_results('models/photos/val/results.txt')

In [ ]:
df_map_photos.sort_values('mAP 50-95').head().style.hide()

In [ ]:
df_map_photos.sort_values('mAP 50-95', ascending=False).head().style.hide()

<img src='models/photos/val/confusion_matrix.png' width='800' alt="Confusion Matrix Photos"/>

Po przedstawionych parametrach widzimy, że model bardzo dobrze poradził sobie z walidacją na danych testowych.

mAP 50-95 na poziomie 95% jest świetnym wynikiem. Jedyną rzeczą mogącą niepokoić jest niższa precyzja na kartach specjalnych (np. Król Czerwo).

Macierz pomyłem obrazuje nam drobne pomyłki w rozpoznawaniu klas lub to czy katę w ogólę wykryto.

#### Videos

In [ ]:
df_map_videos = print_val_results('models/videos/val/results.txt')

In [ ]:
df_map_videos.sort_values('mAP 50-95').head().style.hide()

In [ ]:
df_map_videos.sort_values('mAP 50-95', ascending=False).head().style.hide()

<img src='models/videos/val/confusion_matrix.png' width='800' alt="Confusion Matrix Videos"/>

W przypadku modelu 'videos' sprawa ma się już trochę gorzej. Dalej jest to niezły model, ale nasza precyzja mocno spada (średnio o ok. 5%), co jest zauważalne w maksymalnych wartościach predykcji. Macierz pomyłem pokazuje również, że model trochę się myli.

#### Photos_large

In [ ]:
df_map_photos_large = print_val_results('models/photos_large/val/results.txt')

In [ ]:
df_map_photos_large.sort_values('mAP 50-95').head().style.hide()

In [ ]:
df_map_photos_large.sort_values('mAP 50-95', ascending=False).head().style.hide()

<img src='models/photos_large/val/confusion_matrix.png' width='800' alt="Confusion Matrix Photos Large"/>

Ostatni model 'photos_large' prezentuje się bardzo dobrze. Osiąga największą średnią precyzję oraz tylko dwie klasy są rozpoznawane w mniej niż 90% przypadków (tak samo jak model 'photos').

Macierz pomyłem pokazuję nam niską ilość pomyłek w rozpoznawaniu kart.

Także model ten wydaję się ulepszeniem modelu 'photos'.

## Wybór najlepszego modelu

Po zapoznaniu się z rezultatami wykonanych walidacji przechodzimy do następnego kroku. Musimy zdecydować, którego modelu użyjemy podczas tworzenia aplikacji, czyli w prostych słowach: musimy zdecydować, który model jest najlepszy.

Zacznijmy od spójrzenia na średnie precyzję modeli:

In [16]:
def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "&emsp;&emsp;"
    display(HTML(output))

In [ ]:
display_side_by_side([df_map_photos, df_map_photos_large, df_map_videos], ["'Photos Medium'", "'Photos Large'", "'Videos'"])

### Mutimedia

Po zobaczeniu precyzji danych modeli na poszczególnych kartach, możemy przejść do wykonania predykcji na realnych zdjęciach oraz filmach wideo. W tym celu utworzymy dwie funkcję:
- **predict_test_data** - odpowiada za wykonanie predykcji na wszystkich plikach w danym folderze przez zadany model
- **convert_avi_mp4** - konwertuję pliki wideo z rozszerzeniem *.avi* (domyśle rozszerzenie predykcji na wideo) do *.mp4*. Jest to przydatne do wyświetlania wideo w notatniku

```python
def predict_test_data(model, test_data='../data/test_data', project='predict_test_data', name='predict', img_size = 640):
    if os.path.exists(project):
        shutil.rmtree(project)
        
    results = model(test_data, stream=True, save=True, project=project, name=name, verbose=False, imgsz=img_size)
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs
        

def convert_avi_mp4(folders):
    for fol in folders:
        for video_file in glob(os.path.join(fol)+"/*.avi"):
            moviepy.VideoFileClip(video_file).write_videofile(video_file.replace(".avi", ".mp4"), verbose=False, logger=None)
            os.remove(video_file)

    print('Zakończono konwersję')
```

Teraz tylko pozostało nam wykonać poniższy kod, aby otrzymać nasze predykcje:

```python
model_photos = YOLO('../models/photos/train_2/weights/best.pt')
test_data = '../data/test_data'
project = 'predict_test_data/photos'
name = 'predict'
predict_test_data(model_photos, test_data, project, name)


model_videos = YOLO('../models/videos/train_2/weights/best.pt')
test_data = '../data/test_data'
project = 'predict_test_data/videos'
name = 'predict'
predict_test_data(model_videos, test_data, project, name)


model_photos_large = YOLO('../models/photos_large/train_2/weights/best.pt')
test_data = '../data/test_data'
project = 'predict_test_data/photos_large'
name = 'predict'
predict_test_data(model_photos_large, test_data, project, name)
```

<br></br>
Oraz ten, żeby przekonwertować pliki wideo:

```python
folders = [
    'final_predict/pt/predict',
    'final_predict/onnx/predict',
    'final_predict/tflite/predict',
]

convert_avi_mp4(folders)
```

Teraz zerknijmy jak te predykcje wyglądają.

#### Zdjęcia

<div style="display: flex;">
  <div style="text-align: center; width: 30%;">
    <p style="font-weight: bold; text-align: center;">Photos Medium</p>
    <img src="notebooks/predict_test_data/photos/predict/img_1.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 30%;">
    <p style="font-weight: bold; text-align: center;">Photos Large</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_1.jpg" alt="Image 2" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 30%;">
    <p style="font-weight: bold; text-align: center;">Videos</p>
    <img src="notebooks/predict_test_data/videos/predict/img_1.jpg" alt="Image 3" style="height: 350px;">
  </div>
</div>

&emsp;&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 30%;">
    <img src="notebooks/predict_test_data/photos/predict/img_3.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 30%;">
    <img src="notebooks/predict_test_data/photos_large/predict/img_3.jpg" alt="Image 2" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 30%;">
    <img src="notebooks/predict_test_data/videos/predict/img_3.jpg" alt="Image 3" style="height: 350px;">
  </div>
</div>

&emsp;&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 30%;">
    <img src="notebooks/predict_test_data/photos/predict/img_5.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 30%;">
    <img src="notebooks/predict_test_data/photos_large/predict/img_5.jpg" alt="Image 2" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 30%;">
    <img src="notebooks/predict_test_data/videos/predict/img_5.jpg" alt="Image 3" style="height: 350px;">
  </div>
</div>

&emsp;&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 30%;">
    <img src="notebooks/predict_test_data/photos/predict/img_6.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 30%;">
    <img src="notebooks/predict_test_data/photos_large/predict/img_6.jpg" alt="Image 2" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 30%;">
    <img src="notebooks/predict_test_data/videos/predict/img_6.jpg" alt="Image 3" style="height: 350px;">
  </div>
</div>

Na powyższych obrazach widzimy, że predykcje wykonywane przez model 'videos' odstają dokładnością. Dobitnie widać to na ostatnim zdjęciu, gdzie 'videos' nawet nie rozpoznaje kart.

Po zapoznaniu się z walidacją, tabelą przedstawiającą średnią precyzję danej karty oraz powyższymi predykcjami możemy stwierdzić, że najgorsze wyniki miał model 'videos', więc nie będzie on brał udziału w walce o bycie najlepszym modelem.

Zostają nam modele oparcie na datasecie 'photos':
- *yolov8m* - model 'medium' (średniej wielkości)
- *yolov8l* - model 'large' (duży)

Na pierwszy rzut oka model 'large' wydaję się naturalnym rozwinięciem modelu 'medium', co zdają się potwierdzać wartości metryk. Faktycznie lekko lepiej radzi sobie większy model.

Jednak, żeby mieć pewność zobaczmy jak modele prezentują się na kolejnych przygotowanych zdjęciach oraz filmach:

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="font-weight: bold; text-align: center;">Photos Medium</p>
    <p style="text-align: center;">#1</p>
    <img src="notebooks/predict_test_data/photos/predict/img_2.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="font-weight: bold; text-align: center;">Photos Large</p>
    <p style="text-align: center;">#1</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_2.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Widzimy, że 'Medium' ma problemy z oświetloną kartą oraz prawdopodobieństwa są niższe

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#2</p>
    <img src="notebooks/predict_test_data/photos/predict/img_4.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#2</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_4.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

'Medium' radzi sobie troszkę gorzej, ale widzi jedą predykcję tam, gdzie 'Large' popełnia błąd

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#3</p>
    <img src="notebooks/predict_test_data/photos/predict/img_7.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#3</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_7.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Tutaj widzimy, że jest dość równo, z lekką przewagą 'Large'

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#4</p>
    <img src="notebooks/predict_test_data/photos/predict/img_8.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#4</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_8.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Tutaj możemy zobaczyć, że 'Large' nie jest idealny i myli się w detekcji mocno pochylonej karty ('Medium' też popełnia ten błąd)

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#5</p>
    <img src="notebooks/predict_test_data/photos/predict/img_9.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#5</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_9.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Kolejna zła detekcja od 'Medium'. Tym razem obie nie wykrywają mocno pochylonej karty

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#6</p>
    <img src="notebooks/predict_test_data/photos/predict/img_10.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#6</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_10.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Dobre predykcje, miejscami 'Medium' wykonuję detekcję na niskim prawdopodobieństwie

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#7</p>
    <img src="notebooks/predict_test_data/photos/predict/img_11.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#7</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_11.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Solidne predykcje, nawet jedna karta jest lepiej wykryta na 'Medium'

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#8</p>
    <img src="notebooks/predict_test_data/photos/predict/img_12.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#8</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_12.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Przez padanie światła obydwa modele gubią detekcję, jednak znowu 'Large' wygląda lepiej

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#9</p>
    <img src="notebooks/predict_test_data/photos/predict/img_13.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#9</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_13.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

W obydwu przypadkach bardzo dobre detekcje

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#10</p>
    <img src="notebooks/predict_test_data/photos/predict/img_14.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#10</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_14.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Zdjęcie w ciemniejszym pomieszczeniu pokazuję, że 'Large' w większości przypadków radzi sobie lepiej

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#11</p>
    <img src="notebooks/predict_test_data/photos/predict/img_15.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#11</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_15.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Oba modele dokonują dobrych detekcji, dodatkowo widzą niewidoczne dla sobie nawzajem karty

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#12</p>
    <img src="notebooks/predict_test_data/photos/predict/img_16.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#12</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_16.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Obydwa modele nie widzą jednej karty

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#13</p>
    <img src="notebooks/predict_test_data/photos/predict/img_17.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#13</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_17.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Dobre predykcję, 'Large' widzi mocno pochyloną kartę

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#14</p>
    <img src="notebooks/predict_test_data/photos/predict/img_18.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#14</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_18.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Solidne detekcje

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#15</p>
    <img src="notebooks/predict_test_data/photos/predict/img_19.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#15</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_19.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

W jednym przypadku 'Large' się myli, a 'Medium' nawet nie wykrywa rogu karty

&emsp;

<div style="display: flex;">
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#16</p>
    <img src="notebooks/predict_test_data/photos/predict/img_20.jpg" alt="Image 1" style="height: 350px;">
  </div>
  <div style="text-align: center; width: 40%;">
    <p style="text-align: center;">#16</p>
    <img src="notebooks/predict_test_data/photos_large/predict/img_20.jpg" alt="Image 2" style="height: 350px;">
  </div>
</div>

Prawdiłowe detekcje

Po przeanalizowaniu zdjęć możemy stwierdzić lekką przewagę po stronie modelu 'Photos Large'.

Teraz możemy przejść do analizy nagranych filmów wideo z predykcjami, aby finalnie wybrać najlepszy model.

#### Filmy wideo

In [18]:
def show_videos_html(labels, numbers_in_line, files, description):
  lines = len(files)//numbers_in_line
  html_str = """"""

  if numbers_in_line == 2:
     height = '400px'
     width = '45%'
  elif numbers_in_line == 3:
     height = '300px'
     width = '32%'

  for k in range(lines):
    filepaths = files[k*numbers_in_line : (k+1)*numbers_in_line]
    
    html_str += """<div style="display: flex; font-size: 17px">"""
    for i, file in enumerate(filepaths):
        mp4 = open(file,'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        html_str += """<div style="text-align: center; width: %s;">""" % str(width)
        if k == 0:
          html_str += """<p style="font-weight: bold; text-align: center;">%s</p>""" % labels[i]
        html_str += """<p style="text-align: center;">#%s</p>""" % str(k+1)
        html_str += """
        <video controls style="height: %s;" src="%s"></video>
        """ % (str(height), data_url)
        html_str += """</div>"""
        
    html_str += """</div>"""
    html_str += """<p style="font-size: 17px">%s</p>""" % description[k]
    html_str += """&emsp;"""

  return HTML(html_str)

In [ ]:
labels = ['Photos Medium', 'Photos Large']

numbers_in_line = 2

files = [
    "notebooks/predict_test_data/photos/predict/vid_1.mp4", "notebooks/predict_test_data/photos_large/predict/vid_1.mp4",
    "notebooks/predict_test_data/photos/predict/vid_2.mp4", "notebooks/predict_test_data/photos_large/predict/vid_2.mp4",
    "notebooks/predict_test_data/photos/predict/vid_3.mp4", "notebooks/predict_test_data/photos_large/predict/vid_3.mp4",
    "notebooks/predict_test_data/photos/predict/vid_4.mp4", "notebooks/predict_test_data/photos_large/predict/vid_4.mp4",
    "notebooks/predict_test_data/photos/predict/vid_5.mp4", "notebooks/predict_test_data/photos_large/predict/vid_5.mp4",
    "notebooks/predict_test_data/photos/predict/vid_6.mp4", "notebooks/predict_test_data/photos_large/predict/vid_6.mp4",
    "notebooks/predict_test_data/photos/predict/vid_7.mp4", "notebooks/predict_test_data/photos_large/predict/vid_7.mp4",
    "notebooks/predict_test_data/photos/predict/vid_8.mp4", "notebooks/predict_test_data/photos_large/predict/vid_8.mp4",
    "notebooks/predict_test_data/photos/predict/vid_9.mp4", "notebooks/predict_test_data/photos_large/predict/vid_9.mp4",
    "notebooks/predict_test_data/photos/predict/vid_10.mp4", "notebooks/predict_test_data/photos_large/predict/vid_10.mp4",
    "notebooks/predict_test_data/photos/predict/vid_11.mp4", "notebooks/predict_test_data/photos_large/predict/vid_11.mp4",
    "notebooks/predict_test_data/photos/predict/vid_12.mp4", "notebooks/predict_test_data/photos_large/predict/vid_12.mp4",
    "notebooks/predict_test_data/photos/predict/vid_13.mp4", "notebooks/predict_test_data/photos_large/predict/vid_13.mp4",
    "notebooks/predict_test_data/photos/predict/vid_14.mp4", "notebooks/predict_test_data/photos_large/predict/vid_14.mp4",
]

description = [
    "Oba modele wydają się być bardzo szybkie, choć momentami gubią karty",
    "Tutaj pojawiają się błędy z 'Medium': złe predykcję i brak stabilności",
    "Znowu 'Medium' ma problemy, teraz duża liczba detekcji sprawia problem modelowi",
    "Na tym filmie sprawa ma się już lepiej, zarówno stabilność jak i dokładność są lepsze",
    "Na białym tle oba modele mają problemy, lecz to 'Large' po raz kolejny radzi sobie lepiej",
    "Tutaj oba modele spisują się poniżej oczekiwań",
    "Przy wyższej szybkości przesuwania kamery obydwa modele mają problemy",
    "Szybkość i dokładność predykcji jest tutaj bardzo wysoka",
    "Modele mają gigantyczne problemy z detekcją, gdy jest ciemno",
    "Tak samo jak wyżej. 'Large' widzi tylko kilka kart, przez krótki okres czasu",
    "Przy odpowiednim świetle, predykcje wykonywane na bałym tle są dokładne",
    "Wysoka precyzja i szybkość detekcji dla obu modeli",
    "Ponownie dobrze to wygląda. 'Large' przejawia jednak po raz kolejny lepszą dokładność",
    "Tak samo jak wyżej",
]


show_videos_html(labels, numbers_in_line, files, description)

Możemy zaobserwować, że większy model jest znacznie bardziej stabilny i lepiej radzi sobie z ilością kart na ekranie.

Dlatego wybieramy właśnie model "Photos Large" na ten, który użyjemy w aplikacji.

## Finalny model

Skoro wybraliśmy najlepszy model to przenieśmy go do nowego folderu:

```python
if os.path.exists('../models/final_model'):
    shutil.rmtree('../models/final_model')

shutil.copytree('../models/photos_large/train_2', '../models/final_model')
```

<br></br>
Oraz go zainicjujmy:

```python
final_model = YOLO('../models/final_model/weights/best.pt')
```

#### Konwersja na inne formaty

Mając finalny model, musimy przekonwertować go na odpowiedni format, czyli w naszym przypadku **TFLite** - format umożliwiający nam funkcjonalność na urządzeniach mobilnych.

Przy wykonywaniu tej konwersji otrzymamy jeszcze model w formacie ONNX.

Nie wykonamy jednak jednej konwersji, tylko kilka, w zależności od tego jaką rozdzielczość obrazu akceptuje model:

- 320x320
    ```python
    final_model.export(format='tflite', imgsz=320, half=True)
    ```

- 640x640
    ```python
    final_model.export(format='tflite', imgsz=640, half=True)
    ```

- 1024x1024
    ```python
    final_model.export(format='tflite', imgsz=1024, half=True)
    ```

- 1600x1600
    ```python
    final_model.export(format='tflite', imgsz=1600, half=True)
    ```

Parametr `imgsz` oznacza rozdzielczość obrazu wejściowego do modelu, a `half` odpowiada za kwantyzacje na 16-bit.

Kwantyzacja konwertuje wagi i aktywacje modelu z wysokiej precyzji (jak 32-bitowe liczby zmiennoprzecinkowe) na niższą precyzję (jak 16-bitowe liczby zmiennoprzecinkowe lub 8-bitowe liczby całkowite). Zmniejszając rozmiar modelu, przyspiesza wnioskowanie. Trening z uwzględnieniem kwantyzacji (QAT) to metoda, w której model jest trenowany z uwzględnieniem kwantyzacji, zachowując dokładność lepiej niż kwantyzacja po treningu. Obsługując kwantyzację podczas fazy uczenia, model uczy się dostosowywać do niższej precyzji, utrzymując wydajność przy jednoczesnym zmniejszeniu wymagań obliczeniowych.

Nawet z parametrem `half=True` otrzymujemy dwa modele:
- jeden z precyzją 32-bitową (*best_float32.tflite*)
- drugi z precyzją 16-bitową (*best_float16.tflite*)

Docelowo interesuje nas *best_float16.tflite*, ponieważ waży dwa razy mniej (ok. 85 MB) od *best_float32.tflite* (ok. 170 MB). Jest to dość istotna rzecz w kontekście aplikacji mobilnej.

Przenieśmy nasze modele do folderu *data/export/models/`rozdzielczość`*, np. data/export/models/320x320

#### Predykcje

Zobaczmy jak wcześniejsze predykcje wyglądają na przekonwertowanych modelach. Znowu skorzystamy z funkcji *predict_test_data* oraz *convert_avi_mp4*:

```python
final_model = YOLO('../models/final_model/weights/best.pt')
test_data = '../data/final_test_data'
project = 'final_predict'
name = 'original'
predict_test_data(final_model, test_data, project, name, img_size=1024)


final_model_tflite = YOLO('../data/export/models/320x320/best_float16.tflite')
project = 'final_predict/tflite'
name = '320x320'
predict_test_data(final_model_tflite, test_data, project, name, img_size=320)


final_model_tflite = YOLO('../data/export/models/640x640/best_float16.tflite')
project = 'final_predict/tflite'
name = '640x640'
predict_test_data(final_model_tflite, test_data, project, name, img_size=640)


final_model_tflite = YOLO('../data/export/models/1024x1024/best_float16.tflite')
project = 'final_predict/tflite'
name = '1024x1024'
predict_test_data(final_model_tflite, test_data, project, name, img_size=1024)


final_model_tflite = YOLO('../data/export/models/1600x1600/best_float16.tflite')
project = 'final_predict/tflite'
name = '1600x1600'
predict_test_data(final_model_tflite, test_data, project, name, img_size=1600)





folders = [
    'final_predict/original',
    'final_predict/tflite/320x320',
    'final_predict/tflite/640x640',
    'final_predict/tflite/1024x1024',
    'final_predict/tflite/1600x1600',
]
convert_avi_mp4(folders)
```

##### Zdjęcia

<div style="display: flex;">
  <div style="text-align: center; width: 100%;">
    <p style="font-weight: bold; text-align: center;">#1</p>
    <p style="text-align: center;">Photos Large</p>
    <img src="notebooks/final_predict/original/img_1.jpg" alt="Image 1" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">320x320</p>
    <img src="notebooks/final_predict/tflite/320x320/img_1.jpg" alt="Image 1" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">640x640</p>
    <img src="notebooks/final_predict/tflite/640x640/img_1.jpg" alt="Image 1" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1024x1024</p>
    <img src="notebooks/final_predict/tflite/1024x1024/img_1.jpg" alt="Image 1" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1600x1600</p>
    <img src="notebooks/final_predict/tflite/1600x1600/img_1.jpg" alt="Image 1" style="height: 400px;">
  </div>
</div>
&emsp;&emsp;&emsp;



<div style="display: flex;">
  <div style="text-align: center; width: 100%;">
    <p style="font-weight: bold; text-align: center;">#2</p>
    <p style="text-align: center;">Photos Large</p>
    <img src="notebooks/final_predict/original/img_2.jpg" alt="Image 2" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">320x320</p>
    <img src="notebooks/final_predict/tflite/320x320/img_2.jpg" alt="Image 2" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">640x640</p>
    <img src="notebooks/final_predict/tflite/640x640/img_2.jpg" alt="Image 2" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1024x1024</p>
    <img src="notebooks/final_predict/tflite/1024x1024/img_2.jpg" alt="Image 2" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1600x1600</p>
    <img src="notebooks/final_predict/tflite/1600x1600/img_2.jpg" alt="Image 2" style="height: 400px;">
  </div>
</div>
&emsp;&emsp;



<div style="display: flex;">
  <div style="text-align: center; width: 100%;">
    <p style="font-weight: bold; text-align: center;">#3</p>
    <p style="text-align: center;">Photos Large</p>
    <img src="notebooks/final_predict/original/img_3.jpg" alt="Image 3" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">320x320</p>
    <img src="notebooks/final_predict/tflite/320x320/img_3.jpg" alt="Image 3" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">640x640</p>
    <img src="notebooks/final_predict/tflite/640x640/img_3.jpg" alt="Image 3" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1024x1024</p>
    <img src="notebooks/final_predict/tflite/1024x1024/img_3.jpg" alt="Image 3" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1600x1600</p>
    <img src="notebooks/final_predict/tflite/1600x1600/img_3.jpg" alt="Image 3" style="height: 400px;">
  </div>
</div>
&emsp;&emsp;



<div style="display: flex;">
  <div style="text-align: center; width: 100%;">
    <p style="font-weight: bold; text-align: center;">#4</p>
    <p style="text-align: center;">Photos Large</p>
    <img src="notebooks/final_predict/original/img_4.jpg" alt="Image 4" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">320x320</p>
    <img src="notebooks/final_predict/tflite/320x320/img_4.jpg" alt="Image 4" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">640x640</p>
    <img src="notebooks/final_predict/tflite/640x640/img_4.jpg" alt="Image 4" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1024x1024</p>
    <img src="notebooks/final_predict/tflite/1024x1024/img_4.jpg" alt="Image 4" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1600x1600</p>
    <img src="notebooks/final_predict/tflite/1600x1600/img_4.jpg" alt="Image 4" style="height: 400px;">
  </div>
</div>
&emsp;&emsp;



<div style="display: flex;">
  <div style="text-align: center; width: 100%;">
    <p style="font-weight: bold; text-align: center;">#5</p>
    <p style="text-align: center;">Photos Large</p>
    <img src="notebooks/final_predict/original/img_5.jpg" alt="Image 5" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">320x320</p>
    <img src="notebooks/final_predict/tflite/320x320/img_5.jpg" alt="Image 5" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">640x640</p>
    <img src="notebooks/final_predict/tflite/640x640/img_5.jpg" alt="Image 5" style="height: 400px;">
  </div>
</div>
&emsp;
<div style="display: flex;">
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1024x1024</p>
    <img src="notebooks/final_predict/tflite/1024x1024/img_5.jpg" alt="Image 5" style="height: 400px;">
  </div>
  <div style="text-align: center; width: 50%;">
    <p style="text-align: center;">1600x1600</p>
    <img src="notebooks/final_predict/tflite/1600x1600/img_5.jpg" alt="Image 5" style="height: 400px;">
  </div>
</div>
&emsp;&emsp;





Widzimy, że model **320x320** niezbyt dobrze sobie radzi, ale poza nim wyniki są mocno satysfakcjonujące.

##### Filmy wideo

In [20]:
def show_videos_html_5(numbers_in_line, files, description):
  lines = len(files)//(sum(numbers_in_line))
  first = True

  html_str = """"""

  for k in range(lines):

    for num_in_line in numbers_in_line:
      if num_in_line == 1:
        height = '400px'
        width = '100%'
        labels = ['Photos Large']
        filepaths = files[k*sum(numbers_in_line) : k*sum(numbers_in_line)+1]

      elif num_in_line == 2:
        height = '400px'
        width = '50%'
        if first:
          first = False
          labels = ['320x320', '640x640']
          filepaths = files[k*sum(numbers_in_line)+1 : k*sum(numbers_in_line)+3]
        else:
          first = True
          labels = ['1024x1024', '1600x1600']
          filepaths = files[k*sum(numbers_in_line)+3 : k*sum(numbers_in_line)+5]

      
      html_str += """<div style="display: flex; font-size: 17px">"""
      for i, file in enumerate(filepaths):
          mp4 = open(file,'rb').read()
          data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
          html_str += """<div style="text-align: center; width: %s;">""" % str(width)
          if num_in_line == 1:
            html_str += """<p style="font-weight: bold; text-align: center;">#%s</p>""" % str(k+1)
          html_str += """<p style="text-align: center;">%s</p>""" % labels[i]
          html_str += """
          <video controls style="height: %s;" src="%s"></video>
          """ % (str(height), data_url)
          html_str += """</div>"""
          
      html_str += """</div>"""
      html_str += """<p style="font-size: 17px">%s</p>""" % description[k]
      html_str += """&emsp;"""

    html_str += """&emsp;&emsp;"""

  return HTML(html_str)

In [ ]:
numbers_in_line = (1, 2, 2)

files = [
    "notebooks/final_predict/original/vid_3.mp4",
    "notebooks/final_predict/tflite/320x320/vid_3.mp4", "notebooks/final_predict/tflite/640x640/vid_3.mp4",
    "notebooks/final_predict/tflite/1024x1024/vid_3.mp4", "notebooks/final_predict/tflite/1600x1600/vid_3.mp4",

    # "notebooks/final_predict/original/vid_10.mp4",
    # "notebooks/final_predict/tflite/320x320/vid_10.mp4", "notebooks/final_predict/tflite/640x640/vid_10.mp4",
    # "notebooks/final_predict/tflite/1024x1024/vid_10.mp4", "notebooks/final_predict/tflite/1600x1600/vid_10.mp4",

    "notebooks/final_predict/original/vid_14.mp4",
    "notebooks/final_predict/tflite/320x320/vid_14.mp4", "notebooks/final_predict/tflite/640x640/vid_14.mp4",
    "notebooks/final_predict/tflite/1024x1024/vid_14.mp4", "notebooks/final_predict/tflite/1600x1600/vid_14.mp4",
]

description = [
    "",
    "",
    "",
]


show_videos_html_5(numbers_in_line, files, description)

Jak widzimy różnice w wyeksportowanych modelach względem orginału są minimalne, ba czasami są one lepsze.

### Architektura

Zobaczmy na koniec jak wygląda architektura naszego finalnego modelu:

<div style="text-align: center">
  <img src='notebooks/architecture/final_model_pt.png' width = '90%' alt="Architecture graph"/>
</div>

W tym notatniku przeszliśmy przez najważniejsze kroki podczas tworzenia modelu rozpoznawania obiektów:
1. Utworzenie modelu

2. Uczenie na naszym zbiorze

3. Walidacja przy użyciu zbioru walidacyjnego oraz testowego

4. Predykcja na realnych zdjęciach i filmach wideo

5. Wybór najlepszego modelu spośród kilku

6. Eksport modelu do różnych formatów

7. Test wyeksportowanych modeli

8. Zbadanie architektury modelu


# Podsumowanie

---

Zakończyliśmy pracę w tym notatniku. Teraz pozostało już tylko utworzyć aplikację mobilną.

In [22]:
# Convert
# jupyter nbconvert --to html_embed --Exporter.preprocessors=[\"data.preprocessor.hide_code_preprocessor.HideCodePreprocessor\"] yolov8_models.ipynb